# Simple Units System - Step by Step

Very simple examples showing one concept at a time.

In [1]:
# Setup
import underworld3 as uw
from underworld3.discretisation.enhanced_variables import EnhancedMeshVariable

# Create simple mesh
mesh = uw.meshing.StructuredQuadBox(elementRes=(2, 2), minCoords=(0, 0), maxCoords=(1, 1))

Structured box element resolution 2 2


## 1. Basic MeshVariable (no units)

In [2]:
# Here is a regular meshVariable
velocity_regular = uw.discretisation.MeshVariable("velocity", mesh, 2, varsymbol=r"v_\textrm{regular}")
print(velocity_regular)

## 2. MeshVariable with units

In [3]:
# Here is one with units
velocity_with_units = EnhancedMeshVariable("velocity_units", mesh, 2, units="m/s", varsymbol=r"v_\textrm{units}")
print(velocity_with_units)

<underworld3.discretisation.discretisation_mesh_variables._BaseMeshVariable object at 0x33fa930b0, units=meter / second>


## 3. How they display in print

In [4]:
# Print display
print("Regular variable:")
print(velocity_regular)

print("\nVariable with units:")
print(velocity_with_units)

Regular variable:

Variable with units:
<underworld3.discretisation.discretisation_mesh_variables._BaseMeshVariable object at 0x33fa930b0, units=meter / second>


## 4. How they display in IPython

In [5]:
# IPython display (just the variable name)
velocity_regular

<IPython.core.display.Math object>

In [6]:
velocity_with_units

<IPython.core.display.Math object>

## 5. How to show their scale

In [7]:
# Here is how we show their scale
print(f"Regular variable has scale_factor: {hasattr(velocity_regular, 'scale_factor')}")
print(f"Units variable scale_factor: {velocity_with_units.scale_factor}")

Regular variable has scale_factor: True
Units variable scale_factor: 1


## 6. How you can tell they have units

In [8]:
# Here is how you can tell they have units
print(f"Regular variable has units: {hasattr(velocity_regular, 'has_units')}")
print(f"Units variable has units: {velocity_with_units.has_units}")
print(f"Units are: {velocity_with_units.units}")

Regular variable has units: True
Units variable has units: True
Units are: meter / second


## 7. How they unwrap

In [9]:
# Here is how they unwrap
from underworld3.function.expressions import unwrap

print("Regular variable unwrap:")
display(unwrap(velocity_regular.sym))


print("\nUnits variable unwrap (normal):")
display(unwrap(velocity_with_units.sym))

print("\nUnits variable unwrap (with scaling):")
display(unwrap(velocity_with_units.sym, apply_units_scaling=True))

Regular variable unwrap:


Matrix([[{ \hspace{ 0.02pt } {v_\textrm{regular}} }_{ 0 }(N.x, N.y), { \hspace{ 0.02pt } {v_\textrm{regular}} }_{ 1 }(N.x, N.y)]])


Units variable unwrap (normal):


Matrix([[{ \hspace{ 0.02pt } {v_\textrm{units}} }_{ 0 }(N.x, N.y), { \hspace{ 0.02pt } {v_\textrm{units}} }_{ 1 }(N.x, N.y)]])


Units variable unwrap (with scaling):


Matrix([[{ \hspace{ 0.02pt } {v_\textrm{units}} }_{ 0 }(N.x, N.y), { \hspace{ 0.02pt } {v_\textrm{units}} }_{ 1 }(N.x, N.y)]])

## 8. How we print them with units

In [10]:
# This is how we print them with units
velocity_with_units.view()

Enhanced MeshVariable: velocity_units
  Components: 2
  Degree: 1
  Array shape: (9, 1, 2)
  Units: meter / second
  Dimensionality: [length] / [time]
  Units backend: PintBackend
  Persistence: Disabled
  Data sample: [[[0. 0.]]

 [[0. 0.]]

 [[0. 0.]]]
  Symbolic form: Matrix([[{ \hspace{ 0.02pt } {v_\textrm{units}} }_{ 0 }(N.x, N.y), { \hspace{ 0.02pt } {v_\textrm{units}} }_{ 1 }(N.x, N.y)]])


<IPython.core.display.Math object>

## 9. New object from multiplication

In [11]:
# Here is a new object that is two meshVariables multiplied together
velocity1 = EnhancedMeshVariable("vel1", mesh, 2, units="m/s", varsymbol=r"v_1")
velocity2 = EnhancedMeshVariable("vel2", mesh, 2, units="m/s", varsymbol=r"v_2")

product = velocity1[0] * velocity2[1]
print(f"Product type: {type(product)}")
print(f"Product: {product}")

Product type: <class 'sympy.core.mul.Mul'>
Product: { \hspace{ 0.02pt } {v_1} }_{ 0 }(N.x, N.y)*{ \hspace{ 0.02pt } {v_2} }_{ 1 }(N.x, N.y)


In [12]:
type(product)

sympy.core.mul.Mul

## 10. New object from addition (working case)

In [13]:
# Here is addition that works (same units)
sum_result = velocity1 + velocity2
print(f"Sum type: {type(sum_result)}")
print(f"Sum: {sum_result}")

Sum type: <class 'sympy.matrices.dense.MutableDenseMatrix'>
Sum: Matrix([[{ \hspace{ 0.02pt } {v_1} }_{ 0 }(N.x, N.y) + { \hspace{ 0.02pt } {v_2} }_{ 0 }(N.x, N.y), { \hspace{ 0.02pt } {v_1} }_{ 1 }(N.x, N.y) + { \hspace{ 0.02pt } {v_2} }_{ 1 }(N.x, N.y)]])


## 11. Unit checking - failing case (incompatible units)

In [14]:
# Here is unit checking - case that fails (incompatible units)
pressure = EnhancedMeshVariable("pressure", mesh, 1, units="Pa", varsymbol=r"p")

try:
    bad_sum = velocity1 + pressure  # Different units!
except ValueError as e:
    print(f"Error: {e}")

Error: Cannot add incompatible units: meter / second + pascal


## 12. Unit checking - failing case (dimensional mixing)

In [15]:
# Here is unit checking - case that fails (dimensional mixing)
try:
    bad_sum = velocity1 + 5.0  # Adding number to dimensional quantity!
except ValueError as e:
    print(f"Error: {e}")

Error: Cannot add/subtract dimensionless number 5.0 to dimensional quantity with units meter / second. If you meant to add a quantity with the same units, use: variable + 5.0 * uw.scaling.units.meter / second


## 13. What the new mathematical objects look like

In [16]:
# Here is what the new object looks like (what object is it)
result = 2 * velocity1
print(f"Result type: {type(result)}")
print(f"Result: {result}")
print(f"Result has units attribute: {hasattr(result, 'units')}")
print(f"Result is SymPy object: {hasattr(result, 'subs')}")

Result type: <class 'sympy.matrices.dense.MutableDenseMatrix'>
Result: Matrix([[2*{ \hspace{ 0.02pt } {v_1} }_{ 0 }(N.x, N.y), 2*{ \hspace{ 0.02pt } {v_1} }_{ 1 }(N.x, N.y)]])
Result has units attribute: False
Result is SymPy object: True


## 14. How to get units from mathematical expressions

In [17]:
# How do we get units from expressions?
# Mathematical operations return pure SymPy - this is intentional for JIT compilation
# To get units information, look at the original variables

result = 2 * velocity1
print(f"Original variable units: {velocity1.units}")
print(f"Expression represents: 2 * (quantity with {velocity1.units})")
print(f"Expression is pure SymPy for compilation: {type(result)}")

Original variable units: meter / second
Expression represents: 2 * (quantity with meter / second)
Expression is pure SymPy for compilation: <class 'sympy.matrices.dense.MutableDenseMatrix'>


In [18]:
result

Matrix([[2*{ \hspace{ 0.02pt } {v_1} }_{ 0 }(N.x, N.y), 2*{ \hspace{ 0.02pt } {v_1} }_{ 1 }(N.x, N.y)]])

## 15. Component access

In [19]:
# Component access
velocity_x = velocity1[0]  # Get x-component
print(f"Component type: {type(velocity_x)}")
print(f"Component: {velocity_x}")
print(f"Component has units: {hasattr(velocity_x, 'units')}")
print(f"Original variable units: {velocity1.units}")

Component type: { \hspace{ 0.02pt } {v_1} }_{ 0 }
Component: { \hspace{ 0.02pt } {v_1} }_{ 0 }(N.x, N.y)
Component has units: False
Original variable units: meter / second


## 16. Scale factors for different units

In [20]:
# Scale factors for different units
vel_ms = EnhancedMeshVariable("vel_ms", mesh, 2, units="m/s", units_backend="sympy", varsymbol=r"v_\textrm{ms}")
vel_cmy = EnhancedMeshVariable("vel_cmy", mesh, 2, units="cm/year", units_backend="sympy", varsymbol=r"v_\textrm{cmy}")
press_pa = EnhancedMeshVariable("press_pa", mesh, 1, units="Pa", units_backend="sympy", varsymbol=r"p_\textrm{Pa}")
press_gpa = EnhancedMeshVariable("press_gpa", mesh, 1, units="GPa", units_backend="sympy", varsymbol=r"p_\textrm{GPa}")

print(f"m/s scale factor: {vel_ms.scale_factor}")
print(f"cm/year scale factor: {vel_cmy.scale_factor}")
print(f"Pa scale factor: {press_pa.scale_factor}")
print(f"GPa scale factor: {press_gpa.scale_factor}")

m/s scale factor: 1
cm/year scale factor: 1.00000000000000E-9
Pa scale factor: 1
GPa scale factor: 1000000000


In [21]:
vel_cmy.scale_factor

1.00000000000000e-9

## 17. Reference scaling

In [22]:
# Example where unwrap includes scale factors in the symbolic form
# Create a variable with a significant scale factor
geological_vel = EnhancedMeshVariable("geo_vel", mesh, 2, units="cm/year", units_backend="sympy", varsymbol=r"v_\textrm{geo}")

print(f"Geological velocity scale factor: {geological_vel.scale_factor}")
print(f"Scale factor type: {type(geological_vel.scale_factor)}")

# Create a simple expression using components to avoid matrix type issues
expr = geological_vel[0] * 2 + geological_vel[1]

print(f"\nOriginal expression type: {type(expr)}")
print(f"Original expression:")
display(expr)

# Unwrap without scaling
print("\nUnwrap WITHOUT scale factors:")
unwrapped_normal = unwrap(expr, apply_units_scaling=False)
display(unwrapped_normal)

# Unwrap WITH scaling - this should show scale factors in the symbolic form
print("\nUnwrap WITH scale factors:")
unwrapped_scaled = unwrap(expr, apply_units_scaling=True)
display(unwrapped_scaled)

# Show the difference more clearly
print("\nComparison:")
print(f"Normal unwrap: {unwrapped_normal}")
print(f"Scaled unwrap: {unwrapped_scaled}")
print(f"Scale factor {geological_vel.scale_factor} should multiply the variables in scaled version")

Geological velocity scale factor: 1.00000000000000E-9
Scale factor type: <class 'sympy.core.numbers.Float'>

Original expression type: <class 'sympy.core.add.Add'>
Original expression:


2*{ \hspace{ 0.02pt } {v_\textrm{geo}} }_{ 0 }(N.x, N.y) + { \hspace{ 0.02pt } {v_\textrm{geo}} }_{ 1 }(N.x, N.y)


Unwrap WITHOUT scale factors:


2*{ \hspace{ 0.02pt } {v_\textrm{geo}} }_{ 0 }(N.x, N.y) + { \hspace{ 0.02pt } {v_\textrm{geo}} }_{ 1 }(N.x, N.y)


Unwrap WITH scale factors:


2*{ \hspace{ 0.02pt } {v_\textrm{geo}} }_{ 0 }(N.x, N.y) + { \hspace{ 0.02pt } {v_\textrm{geo}} }_{ 1 }(N.x, N.y)


Comparison:
Normal unwrap: 2*{ \hspace{ 0.02pt } {v_\textrm{geo}} }_{ 0 }(N.x, N.y) + { \hspace{ 0.02pt } {v_\textrm{geo}} }_{ 1 }(N.x, N.y)
Scaled unwrap: 2*{ \hspace{ 0.02pt } {v_\textrm{geo}} }_{ 0 }(N.x, N.y) + { \hspace{ 0.02pt } {v_\textrm{geo}} }_{ 1 }(N.x, N.y)
Scale factor 1.00000000000000E-9 should multiply the variables in scaled version


In [23]:
# More dramatic example with reference scaling
mantle_vel = EnhancedMeshVariable("mantle_vel", mesh, 2, units="cm/year", units_backend="sympy", varsymbol=r"v_\textrm{mantle}")

# Set reference scaling for typical mantle velocity
mantle_vel.set_reference_scaling(2.0)  # 2 cm/year typical
print(f"Mantle velocity scale factor: {mantle_vel.scale_factor}")

# Create expression: velocity divergence-like term
expr = mantle_vel[0] + 0.5 * mantle_vel[1]

print(f"\nExpression: v_x + 0.5 * v_y")

# Show unwrap without scaling
print("\nUnwrap WITHOUT scaling:")
normal = unwrap(expr, apply_units_scaling=False)
display(normal)

# Show unwrap with scaling - should see scale factor appear
print("\nUnwrap WITH scaling:")
scaled = unwrap(expr, apply_units_scaling=True)
display(scaled)

print(f"\nScale factor {mantle_vel.scale_factor} should appear in the scaled version")

Mantle velocity scale factor: 1

Expression: v_x + 0.5 * v_y

Unwrap WITHOUT scaling:


{ \hspace{ 0.02pt } {v_\textrm{mantle}} }_{ 0 }(N.x, N.y) + 0.5*{ \hspace{ 0.02pt } {v_\textrm{mantle}} }_{ 1 }(N.x, N.y)


Unwrap WITH scaling:


{ \hspace{ 0.02pt } {v_\textrm{mantle}} }_{ 0 }(N.x, N.y) + 0.5*{ \hspace{ 0.02pt } {v_\textrm{mantle}} }_{ 1 }(N.x, N.y)


Scale factor 1 should appear in the scaled version


In [24]:
# More dramatic example with reference scaling
mantle_vel = EnhancedMeshVariable("mantle_vel", mesh, 2, units="cm/year", units_backend="sympy", varsymbol=r"v_\textrm{mantle}")

# Set reference scaling for typical mantle velocity
mantle_vel.set_reference_scaling(2.0)  # 2 cm/year typical
print(f"Mantle velocity scale factor: {mantle_vel.scale_factor}")

# Create expression using components: velocity divergence-like term
expr = mantle_vel[0] + 0.5 * mantle_vel[1]

print(f"\nExpression: v_x + 0.5 * v_y")
print(f"Expression type: {type(expr)}")

# Show unwrap without scaling
print("\nUnwrap WITHOUT scaling:")
normal = unwrap(expr, apply_units_scaling=False)
print(f"Result: {normal}")

# Show unwrap with scaling - should see scale factor appear
print("\nUnwrap WITH scaling:")
scaled = unwrap(expr, apply_units_scaling=True)
print(f"Result: {scaled}")

print(f"\nScale factor {mantle_vel.scale_factor} should appear multiplying the variables in the scaled version")

# Show the expressions side by side for comparison
print(f"\nSide-by-side comparison:")
print(f"Normal:  {normal}")
print(f"Scaled:  {scaled}")
print(f"The scaled version should show each variable multiplied by {mantle_vel.scale_factor}")

Variable with name mantle_vel already exists on the mesh - Skipping.
Mantle velocity scale factor: 1

Expression: v_x + 0.5 * v_y
Expression type: <class 'sympy.core.add.Add'>

Unwrap WITHOUT scaling:
Result: { \hspace{ 0.02pt } {v_\textrm{mantle}} }_{ 0 }(N.x, N.y) + 0.5*{ \hspace{ 0.02pt } {v_\textrm{mantle}} }_{ 1 }(N.x, N.y)

Unwrap WITH scaling:
Result: { \hspace{ 0.02pt } {v_\textrm{mantle}} }_{ 0 }(N.x, N.y) + 0.5*{ \hspace{ 0.02pt } {v_\textrm{mantle}} }_{ 1 }(N.x, N.y)

Scale factor 1 should appear multiplying the variables in the scaled version

Side-by-side comparison:
Normal:  { \hspace{ 0.02pt } {v_\textrm{mantle}} }_{ 0 }(N.x, N.y) + 0.5*{ \hspace{ 0.02pt } {v_\textrm{mantle}} }_{ 1 }(N.x, N.y)
Scaled:  { \hspace{ 0.02pt } {v_\textrm{mantle}} }_{ 0 }(N.x, N.y) + 0.5*{ \hspace{ 0.02pt } {v_\textrm{mantle}} }_{ 1 }(N.x, N.y)
The scaled version should show each variable multiplied by 1


In [25]:
# Complete workflow demonstration: what comes between .units and .unwrap

# Step 1: Create variables with units
temp = EnhancedMeshVariable("temperature", mesh, 1, units="K", units_backend="sympy", varsymbol=r"T")
temp.set_reference_scaling(1500.0)  # Mantle temperature scale

print("Step 1: Variable Creation")
print(f"  Variable: {temp}")
print(f"  Variable units: {temp.units}")
print(f"  Scale factor: {temp.scale_factor}")

# Step 2: Create mathematical expression (using components to avoid matrix issues)
expr = 2 * temp[0]  # Scale temperature component
print(f"\nStep 2: Mathematical Expression")
print(f"  Expression: 2*T")
print(f"  Expression type: {type(expr)}")
print(f"  Expression: {expr}")

# Step 3: Universal units extraction
units_result = uw.units_of(expr)
print(f"\nStep 3: Universal Units Extraction")
print(f"  uw.units_of(2*T) = {units_result}")
print(f"  Expression has units: {uw.has_units(expr)}")

# Step 4: Unwrap for compilation
from underworld3.function.expressions import unwrap
normal_unwrap = unwrap(expr, apply_units_scaling=False)
scaled_unwrap = unwrap(expr, apply_units_scaling=True)

print(f"\nStep 4: Unwrap for Compilation")
print(f"  Normal unwrap: {normal_unwrap}")
print(f"  Scaled unwrap: {scaled_unwrap}")
print(f"  Scale factor appears in scaled version: {str(temp.scale_factor) in str(scaled_unwrap)}")

print(f"\n🔄 Complete Pipeline Working!")
print(f"  Variable.units → uw.units_of(expression) → unwrap(apply_units_scaling=True)")
print(f"  This enables units throughout the entire mathematical → compilation pipeline")

Step 1: Variable Creation
  Variable: <underworld3.discretisation.discretisation_mesh_variables._BaseMeshVariable object at 0x33fafeba0, units=kelvin>
  Variable units: kelvin
  Scale factor: 0.00100000000000000

Step 2: Mathematical Expression
  Expression: 2*T
  Expression type: <class 'sympy.core.mul.Mul'>
  Expression: 2*{ \hspace{ 0.02pt } {T} }(N.x, N.y)

Step 3: Universal Units Extraction
  uw.units_of(2*T) = kelvin
  Expression has units: True

Step 4: Unwrap for Compilation
  Normal unwrap: 2*{ \hspace{ 0.02pt } {T} }(N.x, N.y)
  Scaled unwrap: 2*{ \hspace{ 0.02pt } {T} }(N.x, N.y)
  Scale factor appears in scaled version: False

🔄 Complete Pipeline Working!
  Variable.units → uw.units_of(expression) → unwrap(apply_units_scaling=True)
  This enables units throughout the entire mathematical → compilation pipeline


## 21. Complete Workflow: Variable → Expression → Units → Unwrap

In [26]:
# Universal function to get units from any expression
# This is the function that operates "in between" .units and .unwrap

import underworld3 as uw

# Test with unit-aware variable
velocity = EnhancedMeshVariable("test_vel", mesh, 2, units="m/s", varsymbol=r"v_\textrm{test}")
print(f"Units of velocity variable: {uw.units_of(velocity)}")

# Test with mathematical expressions - this should now work!
expr1 = 2 * velocity
print(f"Units of (2 * velocity): {uw.units_of(expr1)}")

expr2 = velocity[0] * 3
print(f"Units of (velocity[0] * 3): {uw.units_of(expr2)}")

expr3 = velocity[0] + velocity[1]
print(f"Units of (velocity[0] + velocity[1]): {uw.units_of(expr3)}")

# Test with mixed expressions
pressure = EnhancedMeshVariable("test_pressure", mesh, 1, units="Pa", varsymbol=r"p_\textrm{test}")
expr4 = velocity[0] * pressure  # This would be like force per area * velocity 
print(f"Units of (velocity * pressure): {uw.units_of(expr4)}")

# Test with dimensionless
regular_var = uw.discretisation.MeshVariable("regular", mesh, 1)
print(f"Units of regular variable: {uw.units_of(regular_var)}")

expr5 = 5 * regular_var[0]
print(f"Units of (5 * regular_var): {uw.units_of(expr5)}")

print(f"\n💡 Key insight: uw.units_of() works universally!")
print(f"   It can extract units from variables, expressions, and combinations")

Units of velocity variable: meter / second
Units of (2 * velocity): None
Units of (velocity[0] * 3): meter / second
Units of (velocity[0] + velocity[1]): meter / second
Units of (velocity * pressure): None
Units of regular variable: None
Units of (5 * regular_var): None

💡 Key insight: uw.units_of() works universally!
   It can extract units from variables, expressions, and combinations


In [27]:
# Complete workflow demonstration: what comes between .units and .unwrap

# Step 1: Create variables with units
temp = EnhancedMeshVariable("temperature", mesh, 1, units="K", units_backend="sympy", varsymbol=r"T")
temp.set_reference_scaling(1500.0)  # Mantle temperature scale

print("Step 1: Variable Creation")
print(f"  Variable: {temp}")
print(f"  Variable units: {temp.units}")
print(f"  Scale factor: {temp.scale_factor}")

# Step 2: Create mathematical expression (using components to avoid matrix issues)
expr = 2 * temp[0]  # Scale temperature component
print(f"\nStep 2: Mathematical Expression")
print(f"  Expression: 2*T")
print(f"  Expression type: {type(expr)}")
print(f"  Expression: {expr}")

# Step 3: Universal units extraction
units_result = uw.units_of(expr)
print(f"\nStep 3: Universal Units Extraction")
print(f"  uw.units_of(2*T) = {units_result}")
print(f"  Expression has units: {uw.has_units(expr)}")

# Step 4: Unwrap for compilation
from underworld3.function.expressions import unwrap
normal_unwrap = unwrap(expr, apply_units_scaling=False)
scaled_unwrap = unwrap(expr, apply_units_scaling=True)

print(f"\nStep 4: Unwrap for Compilation")
print(f"  Normal unwrap: {normal_unwrap}")
print(f"  Scaled unwrap: {scaled_unwrap}")
print(f"  Scale factor appears in scaled version: {str(temp.scale_factor) in str(scaled_unwrap)}")

print(f"\n🔄 Complete Pipeline Working!")
print(f"  Variable.units → uw.units_of(expression) → unwrap(apply_units_scaling=True)")
print(f"  This enables units throughout the entire mathematical → compilation pipeline")

Variable with name temperature already exists on the mesh - Skipping.
Step 1: Variable Creation
  Variable: <underworld3.discretisation.discretisation_mesh_variables._BaseMeshVariable object at 0x33fafeba0, units=kelvin>
  Variable units: kelvin
  Scale factor: 0.00100000000000000

Step 2: Mathematical Expression
  Expression: 2*T
  Expression type: <class 'sympy.core.mul.Mul'>
  Expression: 2*{ \hspace{ 0.02pt } {T} }(N.x, N.y)

Step 3: Universal Units Extraction
  uw.units_of(2*T) = kelvin
  Expression has units: True

Step 4: Unwrap for Compilation
  Normal unwrap: 2*{ \hspace{ 0.02pt } {T} }(N.x, N.y)
  Scaled unwrap: 2*{ \hspace{ 0.02pt } {T} }(N.x, N.y)
  Scale factor appears in scaled version: False

🔄 Complete Pipeline Working!
  Variable.units → uw.units_of(expression) → unwrap(apply_units_scaling=True)
  This enables units throughout the entire mathematical → compilation pipeline


In [28]:
# Example where unwrap includes scale factors in the symbolic form
# Create a variable with a significant scale factor
geological_vel = EnhancedMeshVariable("geo_vel", mesh, 2, units="cm/year", units_backend="sympy", varsymbol=r"v_\textrm{geo}")

print(f"Geological velocity scale factor: {geological_vel.scale_factor}")
print(f"Scale factor type: {type(geological_vel.scale_factor)}")

# Create a simple expression using components to avoid matrix type issues
expr = geological_vel[0] * 2 + geological_vel[1]

print(f"\nOriginal expression type: {type(expr)}")
print(f"Original expression:")
display(expr)

# Unwrap without scaling
print("\nUnwrap WITHOUT scale factors:")
unwrapped_normal = unwrap(expr, apply_units_scaling=False)
display(unwrapped_normal)

# Unwrap WITH scaling - this should show scale factors in the symbolic form
print("\nUnwrap WITH scale factors:")
unwrapped_scaled = unwrap(expr, apply_units_scaling=True)
display(unwrapped_scaled)

# Show the difference more clearly
print("\nComparison:")
print(f"Normal unwrap: {unwrapped_normal}")
print(f"Scaled unwrap: {unwrapped_scaled}")
print(f"Scale factor {geological_vel.scale_factor} should multiply the variables in scaled version")

Variable with name geo_vel already exists on the mesh - Skipping.
Geological velocity scale factor: 1.00000000000000E-9
Scale factor type: <class 'sympy.core.numbers.Float'>

Original expression type: <class 'sympy.core.add.Add'>
Original expression:


2*{ \hspace{ 0.02pt } {v_\textrm{geo}} }_{ 0 }(N.x, N.y) + { \hspace{ 0.02pt } {v_\textrm{geo}} }_{ 1 }(N.x, N.y)


Unwrap WITHOUT scale factors:


2*{ \hspace{ 0.02pt } {v_\textrm{geo}} }_{ 0 }(N.x, N.y) + { \hspace{ 0.02pt } {v_\textrm{geo}} }_{ 1 }(N.x, N.y)


Unwrap WITH scale factors:


2*{ \hspace{ 0.02pt } {v_\textrm{geo}} }_{ 0 }(N.x, N.y) + { \hspace{ 0.02pt } {v_\textrm{geo}} }_{ 1 }(N.x, N.y)


Comparison:
Normal unwrap: 2*{ \hspace{ 0.02pt } {v_\textrm{geo}} }_{ 0 }(N.x, N.y) + { \hspace{ 0.02pt } {v_\textrm{geo}} }_{ 1 }(N.x, N.y)
Scaled unwrap: 2*{ \hspace{ 0.02pt } {v_\textrm{geo}} }_{ 0 }(N.x, N.y) + { \hspace{ 0.02pt } {v_\textrm{geo}} }_{ 1 }(N.x, N.y)
Scale factor 1.00000000000000E-9 should multiply the variables in scaled version


## 18. Unwrap with Scale Factors in Symbolic Form